In [4]:
import numpy as np
import pandas as pd
import openpyxl 
import os
%run ccp_func.ipynb

In [8]:
def bme_colect(quarter):
    #quarter = format: 'Q3-2017'
    
    ccp = 'bme clearing'
    q = quarter.split('-')
    num_q = q[0][1] + q[0][0]
    q = num_q + q[1]

    #basic
    b_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/BME_CLEARING_' + 
                               q.upper() + '.xlsx', sheet_name='AggregatedDataFile')
    #files in use
    a_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/BME_CLEARING_' + 
                               q.upper() + '.xlsx', 
                               sheet_name='BME CLEARING_DataFile_4_4a')
    c_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/BME_CLEARING_' + 
                               q.upper() + '.xlsx', 
                               sheet_name='BME CLEARING_DataFile_6_1')
    d_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/BME_CLEARING_' + 
                               q.upper() + '.xlsx', 
                               sheet_name='BME CLEARING_DataFile_6.2')
    e_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/BME_CLEARING_' + 
                               q.upper() + '.xlsx', 
                               sheet_name='BME CLEARING_DataFile_18_2')
    g_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/BME_CLEARING_' + 
                                   q.upper() + '.xlsx', 
                                   sheet_name='BME CLEARING_DataFile_23')
    
    ex_rate = pd.read_excel('../CCP/data/kursy-walutowe-CCP.xlsx', sheet_name='Arkusz1')
    
     
    f_file = pd.DataFrame({'CCP':ccp.upper()}, index=[0,1,2,3,4])
    f_file['quarter'] = quarter
    f_file['clearning service'] = ['financial derivatives', 'repo', 'power', 'IRS', 'equity']
    b_file = b_file.drop([0]).reset_index() #general description
    
    f_file['4.1.1'] = b_file['4.1.1']
    f_file['4.1.2'] = b_file['4.1.2']
    f_file['4.1.3'] = b_file['4.1.3']
    f_file['4.1.4'] = b_file['4.1.4']
    f_file['4.1.5'] = b_file['4.1.5']
    f_file['4.1.6'] = b_file['4.1.6']
    f_file['4.1.7'] = b_file['4.1.7']
    f_file['4.1.8'] = b_file['4.1.8']
    f_file['4.1.9'] = b_file['4.1.9']
    f_file['4.1.10'] = b_file['4.1.10']
    
    f_file['4.4.7.PDA'] = a_file[a_file.Description == 'PeakDayAmountInPast12Months'].reset_index()['4.4.7']
    f_file['4.4.7.MA'] = a_file[a_file.Description == 'MeanAverageOverPrevious12Months'].reset_index()['4.4.7']
    
    FD_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'BMEC_Financial Derivatives')]['6.1.1'])
    R_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'BMEC_Repo')]['6.1.1'])
    P_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'BMEC_Power')]['6.1.1'])
    I_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'BMEC_IRS')]['6.1.1'])
    E_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'BMEC_Equity')]['6.1.1'])

    f_file['6.1.1.CLIENT'] = [FD_CLIENT, R_CLIENT, P_CLIENT, I_CLIENT, E_CLIENT]
    
    f_file['6.1.1.TOTAL'] = c_file[c_file.Description == 'Total'].reset_index()['6.1.1']

    f_file['6.2.15.PRE'] = d_file[d_file.Description.isin(['TotalIM_PreHaircut'])]['6.2.15'].values
    f_file['6.2.15.POST'] = d_file[d_file.Description.isin(['TotalIM_PostHaircut'])]['6.2.15'].values
        
    f_file['18.1.1.1'] = b_file['18.1.1.1']
    f_file['18.1.1.2'] = b_file['18.1.1.2']

    f_file['18.1.2.2'] = b_file['18.1.2.2']

    f_file['19.1.1'] = b_file['19.1.1']
    f_file['19.1.2'] = b_file['19.1.2']
    
    #23.1.2.OTC & ETD
    FD_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Financial Derivatives'])]['23.1.2'])
    R_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Repo'])]['23.1.2'])
    P_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Power'])]['23.1.2'])
    I_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_IRS'])]['23.1.2'])
    E_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Equity'])]['23.1.2'])

    f_file['23.1.2.OTC'] = [FD_OTC, R_OTC, P_OTC, I_OTC, E_OTC]

    FD_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Financial Derivatives'])]['23.1.2'])
    R_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Repo'])]['23.1.2'])
    P_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Power'])]['23.1.2'])
    I_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_IRS'])]['23.1.2'])
    E_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Equity'])]['23.1.2'])

    f_file['23.1.2.ETD'] = [FD_OTC, R_OTC, P_OTC, I_OTC, E_OTC]

    #23.2.1.OTC & ETD
    FD_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Financial Derivatives'])]['23.2.1'])
    R_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Repo'])]['23.2.1'])
    P_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Power'])]['23.2.1'])
    I_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_IRS'])]['23.2.1'])
    E_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Equity'])]['23.2.1'])

    f_file['23.2.1.OTC'] = [FD_OTC, R_OTC, P_OTC, I_OTC, E_OTC]

    FD_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Financial Derivatives'])]['23.2.1'])
    R_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Repo'])]['23.2.1'])
    P_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Power'])]['23.2.1'])
    I_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_IRS'])]['23.2.1'])
    E_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['BMEC_Equity'])]['23.2.1'])

    f_file['23.2.1.ETD'] = [FD_OTC, R_OTC, P_OTC, I_OTC, E_OTC]
    
    
    ccp_save(ccp, quarter, f_file)

In [9]:
bme_colect('Q1-2016')

ValueError: Length of values does not match length of index